In [4]:
import os

import pandas as pd
import numpy as np

import hopsworks

from datetime import datetime, timedelta
from pytz import timezone

from src.webscraping import (
    activate_web_driver,
    scrape_to_dataframe,
    convert_columns,
    combine_home_visitor,  
    get_todays_matchups,
)

from src.data_processing import (
    process_games,
    add_TARGET,
)

from src.feature_engineering import (
    process_features,
)

from src.hopsworks_utils import (
    save_feature_names,
    convert_feature_names,
)

import json

from pathlib import Path  #for Windows/Linux compatibility
DATAPATH = Path(r'data')

**Load API keys**

In [5]:
from dotenv import load_dotenv

load_dotenv()

try:
    HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']
except:
    raise Exception('Set environment variable HOPSWORKS_API_KEY')

**Activate Webdriver**

In [6]:
# initiate a webdriver in selenium 
# since website data is dynamically generated

driver = activate_web_driver('firefox')

2022-12-15 07:56:47,013 INFO: ====== WebDriver manager ======
2022-12-15 07:56:48,307 INFO: Get LATEST geckodriver version for 108.0 firefox


[WDM] - Downloading: 19.0kB [00:00, 3.90MB/s]                   


2022-12-15 07:56:49,392 INFO: There is no [win64] geckodriver for browser 108.0 in cache
2022-12-15 07:56:49,393 INFO: Getting latest mozilla release info for v0.32.0


[WDM] - Downloading: 19.0kB [00:00, 3.26MB/s]                   

2022-12-15 07:56:49,662 INFO: About to download new driver from https://github.com/mozilla/geckodriver/releases/download/v0.32.0/geckodriver-v0.32.0-win64.zip



[WDM] - Downloading: 100%|██████████| 1.58M/1.58M [00:00<00:00, 5.90MB/s]


2022-12-15 07:56:51,201 INFO: Driver has been saved in cache [C:\Users\Chris\.wdm\drivers\geckodriver\win64\0.32]


**Scrape New Completed Games and Format**

In [7]:

def get_new_games(driver)-> pd.DataFrame:

    # set search for yesterday's games
    DAYS = 1
    SEASON = "" #no season will cause website to default to current season, format is "2022-23"
    TODAY = datetime.now(timezone('EST')) #nba.com uses US Eastern Standard Time
    LASTWEEK = (TODAY - timedelta(days=DAYS))
    DATETO = TODAY.strftime("%m/%d/%y")
    DATEFROM = LASTWEEK.strftime("%m/%d/%y")


    df = scrape_to_dataframe(driver, Season=SEASON, DateFrom=DATEFROM, DateTo=DATETO)

    df = convert_columns(df)

    print(df.info())
    df = combine_home_visitor(df)

    return df

df_new = get_new_games(driver)

# get the SEASON of the last game in the database
# this will used when constructing rows for prediction
SEASON = df_new['SEASON'].max()

df_new




<class 'pandas.core.frame.DataFrame'>
Int64Index: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   HOME            20 non-null     int64         
 1   GAME_DATE_EST   20 non-null     datetime64[ns]
 2   HOME_TEAM_WINS  20 non-null     int64         
 3   PTS             20 non-null     int64         
 4   FG_PCT          20 non-null     float64       
 5   FG3_PCT         20 non-null     float64       
 6   FT_PCT          20 non-null     float64       
 7   REB             20 non-null     int64         
 8   AST             20 non-null     int64         
 9   TEAM_ID         20 non-null     object        
 10  GAME_ID         20 non-null     object        
dtypes: datetime64[ns](1), float64(3), int64(5), object(2)
memory usage: 1.9+ KB
None


,GAME_DATE_EST,HOME_TEAM_WINS,PTS_home,FG_PCT_home,FG3_PCT_home,FT_PCT_home,REB_home,AST_home,HOME_TEAM_ID,GAME_ID,PTS_away,FG_PCT_away,FG3_PCT_away,FT_PCT_away,REB_away,AST_away,VISITOR_TEAM_ID,SEASON
0,2022-12-14,0,119,46.5,42.4,86.2,46,27,1610612744,22200415,125,50.6,41.0,84.0,34,30,1610612754,2022
1,2022-12-14,0,124,50.0,34.3,88.9,39,30,1610612737,22200416,135,50.5,44.4,83.3,46,28,1610612753,2022
2,2022-12-14,1,141,51.6,45.0,75.0,52,25,1610612765,22200414,134,41.2,33.3,79.3,52,21,1610612766,2022
3,2022-12-14,1,124,48.4,37.0,85.0,50,29,1610612758,22200417,123,50.0,28.6,85.2,39,25,1610612761,2022
4,2022-12-14,1,128,60.5,45.0,70.6,40,26,1610612757,22200420,112,46.4,37.9,57.9,40,23,1610612759,2022
5,2022-12-14,1,128,49.5,52.9,80.0,48,25,1610612752,22200418,120,54.8,32.1,86.4,31,25,1610612741,2022
6,2022-12-14,1,110,43.0,42.9,80.0,39,25,1610612748,22200419,108,50.0,37.5,73.9,44,26,1610612760,2022
7,2022-12-14,1,105,52.6,33.3,78.9,43,26,1610612739,22200421,90,39.2,34.2,73.1,33,17,1610612742,2022
8,2022-12-14,0,128,52.7,48.7,56.5,24,40,1610612764,22200422,141,65.1,25.0,82.9,45,31,1610612743,2022
9,2022-12-14,0,88,41.9,18.2,73.3,43,15,1610612750,22200423,99,38.2,30.4,81.0,52,21,1610612746,2022


**Retrieve todays games**

In [8]:
#retrieve list of teams playing today

# get today's games on NBA schedule
teams_list = get_todays_matchups(driver)

# example output:
# ['/team/1610612759/spurs/', '/team/1610612748/heat/',...

print(teams_list)

# create list of matchups by parsing out team ids from teams_list
# second team id is always the home team
team_count = len(teams_list) 
matchups = []
for i in range(0,team_count,2):
    visitor_id = teams_list[i].partition("team/")[2].partition("/")[0] #extract team id from text
    home_id = teams_list[i+1].partition("team/")[2].partition("/")[0]
    matchups.append([visitor_id, home_id])

matchups

['/team/1610612765/pistons/', '/team/1610612766/hornets/', '/team/1610612744/warriors/', '/team/1610612754/pacers/', '/team/1610612737/hawks/', '/team/1610612753/magic/', '/team/1610612758/kings/', '/team/1610612761/raptors/', '/team/1610612752/knicks/', '/team/1610612741/bulls/', '/team/1610612748/heat/', '/team/1610612760/thunder/', '/team/1610612757/blazers/', '/team/1610612759/spurs/', '/team/1610612739/cavaliers/', '/team/1610612742/mavericks/', '/team/1610612764/wizards/', '/team/1610612743/nuggets/', '/team/1610612750/timberwolves/', '/team/1610612746/clippers/']


[['1610612765', '1610612766'],
 ['1610612744', '1610612754'],
 ['1610612737', '1610612753'],
 ['1610612758', '1610612761'],
 ['1610612752', '1610612741'],
 ['1610612748', '1610612760'],
 ['1610612757', '1610612759'],
 ['1610612739', '1610612742'],
 ['1610612764', '1610612743'],
 ['1610612750', '1610612746']]

**Close Webdriver**

In [9]:
driver.close() 

**Create Blank Rows**

In [14]:
# append today's matchups to the new games dataframe

# since we don't have access to official game ids, we will use the format 20MMDD00 + index as a game id
# This format is used to insure that feature engineering will treat this as a regular season game
# and to be able to find these incomplete games in the database
# this data will be updated later after the game is played

DUMMY_GAME_ID = int("20" + datetime.now(timezone('EST')).strftime("%m%d") + "00")

df_today = df_new.drop(df_new.index) #empty copy of df_new with same columns
for i, matchup in enumerate(matchups):
    game_details = {'HOME_TEAM_ID': matchup[1], 
                    'VISITOR_TEAM_ID': matchup[0], 
                    'GAME_DATE_EST': datetime.now(timezone('EST')).strftime("%Y-%m-%d"), 
                    'GAME_ID': DUMMY_GAME_ID+i,
                    'SEASON': SEASON,
                    } 
    game_details_df = pd.DataFrame(game_details, index=[i])
    # append to new games dataframe
    df_today = pd.concat([df_today, game_details_df], ignore_index = True)

#blank rows will be filled with 0 to prevent issues with feature engineering
df_today = df_today.fillna(0) 

df_today


,GAME_DATE_EST,HOME_TEAM_WINS,PTS_home,FG_PCT_home,FG3_PCT_home,FT_PCT_home,REB_home,AST_home,HOME_TEAM_ID,GAME_ID,PTS_away,FG_PCT_away,FG3_PCT_away,FT_PCT_away,REB_away,AST_away,VISITOR_TEAM_ID,SEASON
0,2022-12-15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1610612766,20121500,0.0,0.0,0.0,0.0,0.0,0.0,1610612765,2022
1,2022-12-15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1610612754,20121501,0.0,0.0,0.0,0.0,0.0,0.0,1610612744,2022
2,2022-12-15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1610612753,20121502,0.0,0.0,0.0,0.0,0.0,0.0,1610612737,2022
3,2022-12-15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1610612761,20121503,0.0,0.0,0.0,0.0,0.0,0.0,1610612758,2022
4,2022-12-15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1610612741,20121504,0.0,0.0,0.0,0.0,0.0,0.0,1610612752,2022
5,2022-12-15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1610612760,20121505,0.0,0.0,0.0,0.0,0.0,0.0,1610612748,2022
6,2022-12-15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1610612759,20121506,0.0,0.0,0.0,0.0,0.0,0.0,1610612757,2022
7,2022-12-15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1610612742,20121507,0.0,0.0,0.0,0.0,0.0,0.0,1610612739,2022
8,2022-12-15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1610612743,20121508,0.0,0.0,0.0,0.0,0.0,0.0,1610612764,2022
9,2022-12-15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1610612746,20121509,0.0,0.0,0.0,0.0,0.0,0.0,1610612750,2022


**Access Feature Store**

In [ ]:
project = hopsworks.login(api_key_value=HOPSWORKS_API_KEY)
fs = project.get_feature_store()

**Access Feature Group**

In [ ]:
rolling_stats_fg = fs.get_feature_group(
    name="rolling_stats",
    version=1,
)

**Query Old Data Needed for Feature Engineering of New Data**

To generate features like rolling averages for the new games, older data from previous games is needed since some of the rolling averages might extend back 15 or 20 games or so.

In [ ]:
BASE_FEATURES = ['game_date_est',
 'game_id',
 'home_team_id',
 'visitor_team_id',
 'season',
 'pts_home',
 'fg_pct_home',
 'ft_pct_home',
 'fg3_pct_home',
 'ast_home',
 'reb_home',
 'pts_away',
 'fg_pct_away',
 'ft_pct_away',
 'fg3_pct_away',
 'ast_away',
 'reb_away',
 'home_team_wins',
]

ds_query = rolling_stats_fg.select(BASE_FEATURES)
df_old = ds_query.read()
df_old


**Convert Feature Names back to original mixed case**

In [ ]:
df_old = convert_feature_names(df_old)
df_old

**Update Yesterday's Matchup Predictions with New Final Results**

In [ ]:
# filter out games that are pending final results
# (these were the rows used for prediction yesterday)
# and then update these with the new results
#>>> df1.set_index('Code', inplace=True)
#>>> df1.update(df2.set_index('Code'))
#>>> df1.reset_index()  # to recover the initial structure

df_pending = df_old[df_old['GAME_ID'] < 20000100]

df_pending.update(df_new)

df_pending

**Add Today's Matchups for Feature Engineering**

In [ ]:
df_combined = pd.concat([df_combined, df_today], ignore_index = True)
df_combined

**Data Processing**

In [ ]:
df_combined = process_games(df_combined) 
df_combined = add_TARGET(df_combined)
df_combined

**Feature Engineering**

In [ ]:
# Feature engineering to add: 
    # rolling averages of key stats, 
    # win/lose streaks, 
    # home/away streaks, 
    # specific matchup (team X vs team Y) rolling averages and streaks

df_combined = process_features(df_combined)
df_combined


**Insert New Data into Feature Group**

In [ ]:

def test():
    # retrieve only new games from the combined dataframe now that feature engineering is complete

    # set index to GAME_ID
    df_combined = df_combined.set_index('GAME_ID')
    df_new = df_new.set_index('GAME_ID')
    
    # retrieve only new games
    df_new = df_combined.loc[df_new.index]

    # reset GAME_ID index back to column
    df_new = df_new.reset_index()

    # convert certain features back to int32 for Hopsworks compatibility
    df_new['GAME_ID'] = df_new['GAME_ID'].astype('int32')
    df_new['HOME_TEAM_WINS'] = df_new['HOME_TEAM_WINS'].astype('int32')
    df_new['TARGET'] = df_new['TARGET'].astype('int32')

    # save new games to Hopsworks feature group
    rolling_stats_fg.insert(df_new, write_options={"wait_for_job" : False})

    df_new 

rolling_stats_fg.insert(df_combined, write_options={"wait_for_job" : False})